In [4]:
from MOASMO_parameter_allbasin_emulator import *

In [11]:
    infile_basin_info = f"/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv"
    infile_param_info = '/glade/u/home/guoqiang/CTSM_repos/CTSM_calibration/src/parameter/CTSM_CAMELS_calibparam_2410.csv'
    infile_attr_foruse = '/glade/u/home/guoqiang/CTSM_repos/CTSM_calibration/data/camels_attributes_table_TrainModel.csv'
    inpath_moasmo = "/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator"
    outpathname = "emulator_CV_test"
    path_CTSM_case = f'/glade/work/guoqiang/CTSM_CAMELS/Calib_HH_emulator'
    train_index = np.setdiff1d(np.arange(627), np.arange(0, 627, 5))
    numruns=100
    objfunc='normKGE'
    ncpus = 1
    iterend = 1
    suffix = 'CV1'
    suffix_defa_source = 'LSEnormKGE' # temporary suffix if CV1 has not been generated


    outpath = f"{inpath_moasmo}/{outpathname}"
    os.makedirs(outpath, exist_ok=True)

    # Load data: same for all iterations
    df_basin_info = pd.read_csv(infile_basin_info)
    df_basin_info.index = np.arange(len(df_basin_info))
    all_index = np.arange(len(df_basin_info))

    test_index = np.setdiff1d(all_index, train_index)

    # information for all basins
    df_param_info = pd.read_csv(infile_param_info)

    file_defa_param = f'{outpath}/camels_ctsm_defa_param_train_{suffix}.csv'
    df_param_defa_train = read_allbasin_defa_params(path_CTSM_case, infile_param_info, file_defa_param, train_index)
    file_defa_param = f'{outpath}/camels_ctsm_defa_param_test_{suffix}.csv'
    df_param_defa_test = read_allbasin_defa_params(path_CTSM_case, infile_param_info, file_defa_param, test_index)

    file_param_lb = f'{outpath}/camels_ctsm_all_param_lb_train_{suffix}.gz'
    file_param_ub = f'{outpath}/camels_ctsm_all_param_ub_train_{suffix}.gz'
    
    df_param_lb_train, df_param_ub_train = load_basin_param_bounds(inpath_moasmo, df_param_defa_train, file_param_lb, file_param_ub, train_index, suffix_defa_source)
    file_param_lb = f'{outpath}/camels_ctsm_all_param_lb_test_{suffix}.gz'
    file_param_ub = f'{outpath}/camels_ctsm_all_param_ub_test_{suffix}.gz'
    df_param_lb_test, df_param_ub_test = load_basin_param_bounds(inpath_moasmo, df_param_defa_test, file_param_lb, file_param_ub, test_index, suffix_defa_source)

    
    file_camels_attribute = f'{outpath}/camels_basin_attribute_train_{suffix}.pkl'
    df_att_train = read_camels_attributes(infile_basin_info, file_camels_attribute, train_index)
    file_camels_attribute = f'{outpath}/camels_basin_attribute_test_{suffix}.pkl'
    df_att_test = read_camels_attributes(infile_basin_info, file_camels_attribute, test_index)

    df_att_foruse = pd.read_csv(infile_attr_foruse)
    useattrs = list(df_att_foruse[df_att_foruse['att_Xie2021'].values]['Attribute_text'].values)
    print("The number of attributes used:", len(useattrs))
    print(useattrs)



File exists: /glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/emulator_CV_test/camels_basin_attribute_train_CV1.pkl
File exists: /glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/emulator_CV_test/camels_basin_attribute_test_CV1.pkl
The number of attributes used: 27
['mean_elev', 'mean_slope', 'area_gauges2', 'p_mean', 'pet_mean', 'aridity', 'p_seasonality', 'frac_snow', 'high_prec_freq', 'high_prec_dur', 'low_prec_freq', 'low_prec_dur', 'frac_forest', 'lai_max', 'lai_diff', 'dom_land_cover', 'dom_land_cover_frac', 'soil_depth_pelletier', 'soil_depth_statsgo', 'soil_porosity', 'soil_conductivity', 'max_water_content', 'sand_frac', 'silt_frac', 'clay_frac', 'carbonate_rocks_frac', 'geol_permeability']


In [12]:
    # Load data: outputs from each iteration from training basins
    for iter in range(0, iterend):
        file_all_param = f'{outpath}/camels_ctsm_all_param_train_{suffix}_iter{iter}.gz'
        file_all_metric = f'{outpath}/camels_ctsm_all_metric_train_{suffix}_iter{iter}.gz'
        file_all_basinid = f'{outpath}/camels_ctsm_all_basinid_train_{suffix}_iter{iter}.gz'

        df_param_i, df_metric_i, df_basinid_i = load_all_basin_params_metrics(inpath_moasmo, df_param_defa_train,
                                                                              df_basin_info, iter, file_all_param,
                                                                              file_all_metric, file_all_basinid,
                                                                              train_index, suffix_defa_source)

        df_basinid_i['iter'] = iter

        if iter == 0:
            df_param = df_param_i
            df_metric = df_metric_i
            df_basinid = df_basinid_i
        else:
            df_param = pd.concat([df_param, df_param_i])
            df_metric = pd.concat([df_metric, df_metric_i])
            df_basinid = pd.concat([df_basinid, df_basinid_i])

    df_param.index = np.arange(len(df_param))
    df_metric.index = np.arange(len(df_metric))
    df_basinid.index = np.arange(len(df_basinid))

    index = np.isnan(np.sum(df_metric.values, axis=1) + np.sum(df_param.values, axis=1))
    df_param = df_param[~index]
    df_metric = df_metric[~index]
    df_basinid = df_basinid[~index]

    df_param.index = np.arange(len(df_param))
    df_metric.index = np.arange(len(df_metric))
    df_basinid.index = np.arange(len(df_basinid))

    print('Number of nan samples:', np.sum(index))
    print("Number of original parameter sets:", len(index))
    print("Number of final parameter sets:", len(df_param))




Number of nan samples: 1079
Number of original parameter sets: 100200
Number of final parameter sets: 99121


In [13]:
    # One-hot encoding for categorical attributes
    df_att = pd.concat([df_att_train, df_att_test])
    df_att.index = np.arange(len(df_att))
    df_att_use = df_att[useattrs + ["hru_id"]]
    for att in useattrs:
        if df_att_use[att].dtype == "object":
            print('Convert', att, 'to one-hot encoding')
            enc = OneHotEncoder(sparse=False)
            enc.fit(df_att_use[[att]])
            encnames = [att + "_" + str(i) for i in range(len(enc.categories_[0]))]
            print('New columns:', encnames)
            df_enc = pd.DataFrame(enc.transform(df_att_use[[att]]), columns=encnames)
            df_att_use = pd.concat([df_att_use, df_enc], axis=1)
            df_att_use = df_att_use.drop([att], axis=1)

    df_att_use_train = df_att_use[:len(df_att_train)]
    df_att_use_test = df_att_use[:len(df_att_test)]
    df_att_use_train.index = np.arange(len(df_att_use_train))
    df_att_use_test.index = np.arange(len(df_att_use_test))
    
    useattrs = list(df_att_use_train.columns)
    useattrs.remove('hru_id')

    # Prepare model input and output
    df_input = df_param.copy()
    df_input["hru_id"] = df_basinid["hru_id"]
    df_input = df_input.merge(df_att_use_train[useattrs + ["hru_id"]], on="hru_id", how="left")
    df_input = df_input.drop(["hru_id"], axis=1)

    inputnames = list(df_param.columns) + useattrs
    x_all = df_input[inputnames].values.copy()
    print("Input shape:", x_all.shape)

    print('Train/test model')
    print('Train index:', train_index)



Convert dom_land_cover to one-hot encoding
New columns: ['dom_land_cover_0', 'dom_land_cover_1', 'dom_land_cover_2', 'dom_land_cover_3', 'dom_land_cover_4', 'dom_land_cover_5', 'dom_land_cover_6', 'dom_land_cover_7', 'dom_land_cover_8', 'dom_land_cover_9', 'dom_land_cover_10', 'dom_land_cover_11']
Input shape: (99121, 54)
Train/test model
Train index: [  1   2   3   4   6   7   8   9  11  12  13  14  16  17  18  19  21  22
  23  24  26  27  28  29  31  32  33  34  36  37  38  39  41  42  43  44
  46  47  48  49  51  52  53  54  56  57  58  59  61  62  63  64  66  67
  68  69  71  72  73  74  76  77  78  79  81  82  83  84  86  87  88  89
  91  92  93  94  96  97  98  99 101 102 103 104 106 107 108 109 111 112
 113 114 116 117 118 119 121 122 123 124 126 127 128 129 131 132 133 134
 136 137 138 139 141 142 143 144 146 147 148 149 151 152 153 154 156 157
 158 159 161 162 163 164 166 167 168 169 171 172 173 174 176 177 178 179
 181 182 183 184 186 187 188 189 191 192 193 194 196 197 198 1

In [16]:
    if objfunc == 'normKGE':
        print('Use normalized KGE as output')
        df_output = df_metric.copy()
        y_all = df_output[["kge"]].values.copy()
        y_all = y_all / (2 - y_all)  # Normalize KGE

        # Train a random forest emulator
        outfile = f'{outpath}/RF_emulator_for_iter{iterend}_{suffix}.pkl'
        if os.path.isfile(outfile):
            with open(outfile, 'rb') as file:
                em_model = pickle.load(file)
        else:
            modelconfig = {'n_estimators': 100, 'random_state': 42, 'max_depth': 40}
            em_model = RandomForestRegressor(**modelconfig, n_jobs=ncpus)
            em_model.fit(x_all[::100,:], y_all[::100])
            with open(outfile, 'wb') as file:
                pickle.dump(em_model, file)


    elif objfunc == 'norm2err':
        print('Use normalized mae and mmae as output')
        # normalization is performed for each basin
        df_output = df_metric.copy()
        metvalues = df_output[['mae', 'max_mon_abs_err']].values.copy()
        y_all = np.nan * metvalues
        for i in range(len(train_index)):
            indi = df_basinid['basin_id'].values == train_index[i]
            di = metvalues[indi, :]
            di = (di - np.nanmin(di, axis=0)) / (np.nanmax(di, axis=0) - np.nanmin(di, axis=0))
            y_all[indi, :] = di

        # Train a random forest emulator
        outfile = f'{outpath}/RF_emulator_2errOBJfunc_for_iter{iterend}_{suffix}.pkl'
        outfile_eval = f'{outpath}/RF_emulator_2errOBJfunc_for_iter{iterend}_{suffix}_eval.npz'
        if os.path.isfile(outfile):
            with open(outfile, 'rb') as file:
                em_model = pickle.load(file)
        else:
            modelconfig = {'n_estimators': 200, 'random_state': 42, 'max_depth': 40}
            em_model = RandomForestRegressor(**modelconfig, n_jobs=ncpus)
            em_model.fit(x_all, y_all)
            with open(outfile, 'wb') as file:
                pickle.dump(em_model, file)




Use normalized KGE as output


In [19]:
    param_names = df_param_info['Parameter'].values
    parallel_process_basins(df_basinid, df_param_lb_train, df_param_ub_train,
                            x_all, df_input, y_all,
                            param_names, inputnames, em_model, inpath_moasmo, ncpus, numruns, iterend, train_index, suffix)



num_objfunc: 1
Error processing basin 1-0: [Errno 2] No such file or directory: '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/level1_1_calib/param_sets_CV1/paramset_iter0_trial0.pkl'
num_objfunc: 1


Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/glade/work/guoqiang/conda-envs/npl-2024a-tgq/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/glade/work/guoqiang/conda-envs/npl-2024a-tgq/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/glade/work/guoqiang/conda-envs/npl-2024a-tgq/lib/python3.11/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^


KeyboardInterrupt: 

In [35]:
    #### predict parameter in unseen basins
    suffixtest = suffix+'test'
    numruns_test = numruns # can be smaller
    if objfunc == 'normKGE':
        num_objfunc=1
    else:
        sys.exit('Not tested objfunc')

    df_att_use_test2 = df_att_use_test.drop(['hru_id'], axis=1)

    xlb_mean_test = np.nan * np.zeros([len(df_param_lb_test), len(inputnames)])
    xub_mean_test = np.nan * np.zeros([len(df_param_ub_test), len(inputnames)])   
    for i in range(len(df_param_lb_test)):
        param_lb_mean = df_param_lb_test.values[i, :]
        param_ub_mean = df_param_ub_test.values[i, :]
        attrvalues = df_att_use_test2.values[i,:]
        xlb_mean_test[i,:] = np.hstack([param_lb_mean, attrvalues])
        xub_mean_test[i,:] = np.hstack([param_ub_mean, attrvalues])
    
    parallel_process_basins_predictunseen(xlb_mean_test, xub_mean_test, param_names, em_model, inpath_moasmo, ncpus, numruns_test, iterend, test_index, suffixtest, num_objfunc)


Error processing basin 0-0: [Errno 2] No such file or directory: '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/level1_0_calib/param_sets_CV1test/paramset_iter0_trial0.pkl'
Error processing basin 5-1: [Errno 2] No such file or directory: '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/level1_5_calib/param_sets_CV1test/paramset_iter0_trial0.pkl'
Error processing basin 10-2: [Errno 2] No such file or directory: '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/level1_10_calib/param_sets_CV1test/paramset_iter0_trial0.pkl'
Error processing basin 15-3: [Errno 2] No such file or directory: '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/level1_15_calib/param_sets_CV1test/paramset_iter0_trial0.pkl'
Error processing basin 20-4: [Errno 2] No such file or directory: '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/level1_20_calib/param_sets_CV1test/paramset_it

Process ForkPoolWorker-3:

KeyboardInterrupt

